# Q-Learning with TensorFlow

We will create a q-Learning algorithm to learn how to play 'Connect 3'. Similar to the famous Connect 4, but with far fewer possible states.

Here is the size of the 'Connect 3' board (4x4) board:

| Col 1 | Col 2 | Col 3 | Col 4 |
|-|-|-|-|
|-|-|-|-|
|-|-|-|-|
|-|-|-|-|
|-|-|-|-|

We will denote a board with x's and o's.  A potential board could look like:

| Col 1 | Col 2 | Col 3 | Col 4 |
|-|-|-|-|
| |x| | |
| |o| | |
|o|x| | |
|x|x|o|o|

We will store the board states as a dictionary, with the board being stored as key and the row of the reward matrix being the key.

For example, if we observe the above state of the board, and we have not observed it before, we add another key to the dictionary, 'eeeeexeeeoeeoxeexxoe' where 'e' stands for empty, and the key is read in reading order (top left to bottom right).  All keys will be 16 letters long.

```python
board_dict['eeeeexeeeoeeoxeexxoo'] = reward_matrix.shape[0] + 1
new_row = [0, 0, 0, 0]
reward_matrix = np.vstack([reward_matrix, new_row])
```

Note that the `new_row` vector only has four possible reward states because there are only four possible actions. These actions are to drop your piece in column 1, 2, 3, or 4.

### Q Learning Methodology
------------------

Q-learning relies on a table lookup.  That table is called a Q-table and the usage of it is to lookup the current state, and take the action that has the highest Q-value.

In other words, Q-learning methodology is essentially looking up the current state in the Q-table, and choosing the action that has the highest Q value to progress onto the next state.

We also update the Q table according to the following equation:

$$Q(s_{i},a_{j}) = r + \gamma \left( \max_{a \in A_{i+1}} Q(s_{i+1}, a) \right)$$

Where $A_{i+1}$ is set of all possible actions in state $s_{i+1}$.  In other words, our updated value in position $(s_{i}, a_{j})$ in the Q-table is a reward, $r$, plus a discounted Q-value for the next step.  The discounted amount is given by $\gamma$, which is a positive fraction less than one (a common value is $\gamma=0.99$).

### This Simulation
-------------------------

For this specific game simulation, we should define some specifics.  The reward, $r$, for a move will be zero, unless we achieve four in a row, then the reward will be 1.

While it may be tempting to make the reward proportional to how many we can get in a row (e.g. make the reward 0.5 for getting two in a row), doing this may force the algorithm to adopt a non-optimal strategy.  For example, maybe it is better to try to connect two pieces separated by 1 space than to build from 1 to 2 to 3 in a row.

When the algorithm does get a win, eventually the reward will propagate backwards to decisions that have lead it to win.

We set the descounted gain as $\gamma = 0.99$ and the learning rate (which is how much we change the current Q values, to $\eta = 0.5$.

In [30]:
import numpy as np

In [31]:
t = np.array([[1,2],[3,1]])

In [35]:
np.power(3, 16)

43046721

In [2]:
env = gym.make('FrozenLake-v0')

[2017-02-20 15:06:55,856] Making new env: FrozenLake-v0


In [15]:
#Initialize table with all zeros
Q = np.zeros([env.observation_space.n,env.action_space.n])
# Set learning parameters
lr = .85
y = .99
num_episodes = 2000
#create lists to contain total rewards and steps per episode
jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    #The Q-Table learning algorithm
    while j < 99:
        j+=1
        #Choose an action by greedily (with noise) picking from Q table
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #Get new state and reward from environment
        s1,r,d,_ = env.step(a)
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
        if d == True:
            break
    jList.append(j)
    rList.append(rAll)

In [6]:
print("Score over time: {}".format(sum(rList)/num_episodes))
print("Final Q-Table Values: {}".format(Q))

Score over time: 0.4765
Final Q-Table Values: [[  1.27149803e-02   6.67873088e-01   7.88563722e-03   3.59903163e-03]
 [  1.63838130e-03   1.64431327e-03   2.46295778e-04   4.46821192e-01]
 [  2.02305140e-04   3.76659172e-03   8.21697654e-03   2.50961821e-01]
 [  1.30828329e-04   9.08831267e-04   8.90749479e-04   1.55891436e-01]
 [  4.18951127e-01   2.37370694e-02   2.47074568e-03   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  2.06599254e-05   9.91590650e-06   1.69375005e-04   8.74995582e-05]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  5.18448283e-03   6.07880612e-03   0.00000000e+00   7.95344009e-01]
 [  0.00000000e+00   9.02184277e-01   2.36259318e-03   0.00000000e+00]
 [  9.67735898e-01   0.00000000e+00   0.00000000e+00   7.64250441e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000

In [17]:
s = env.reset()
s

0

In [23]:
env.render()

SFFF
FHFH
FFFH
HFFG
  (Up)


In [27]:
next_s, r, is_end, inf = env.step(action=3)
print(next_s, r, is_end, inf)
env.render()

0 0.0 False {'prob': 0.3333333333333333}
SFFF
FHFH
FFFH
HFFG
  (Up)


In [11]:
[x for x in env]

TypeError: 'FrozenLakeEnv' object is not iterable